In [1]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[1]") \
                    .appName('DataFrames') \
                    .getOrCreate()

In [2]:
SpContext = spark.sparkContext

In [3]:
inputPath = "auto-data.csv"
lines = SpContext.textFile(inputPath)
header = lines.first()
datalines = lines.filter(lambda x: x!=header)
print("datalines.count()", datalines.count())

datalines.count() 197


In [4]:
from pyspark.sql import Row

parts = datalines.map(lambda l: l.split(","))
autoMap = parts.map(lambda p: Row(make=p[0],body=p[4],hp=int(p[7])))
autoMap.take(3)

[Row(body='hatchback', hp=69, make='subaru'),
 Row(body='hatchback', hp=48, make='chevrolet'),
 Row(body='hatchback', hp=68, make='mazda')]

In [5]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(SpContext)

autoDf = sqlContext.createDataFrame(autoMap)
autoDf.show()

+---------+---+----------+
|     body| hp|      make|
+---------+---+----------+
|hatchback| 69|    subaru|
|hatchback| 48| chevrolet|
|hatchback| 68|     mazda|
|hatchback| 62|    toyota|
|hatchback| 68|mitsubishi|
|hatchback| 60|     honda|
|    sedan| 69|    nissan|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 68|     mazda|
|hatchback| 68|mitsubishi|
|hatchback| 68|     dodge|
|hatchback| 68|  plymouth|
|hatchback| 70| chevrolet|
|hatchback| 62|    toyota|
|hatchback| 68|     dodge|
|hatchback| 58|     honda|
|hatchback| 62|    toyota|
|hatchback| 76|     honda|
|    sedan| 70| chevrolet|
+---------+---+----------+
only showing top 20 rows



#### Crear dataframe directamente desde CSV

In [6]:
autoDf1 = sqlContext.read.format('csv').options(header='True').load(inputPath)
autoDf1.show()

+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|      MAKE|FUELTYPE|ASPIRE|DOORS|     BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+----------+--------+------+-----+---------+-----+---------+---+----+--------+-------+-----+
|    subaru|     gas|   std|  two|hatchback|  fwd|     four| 69|4900|      31|     36| 5118|
| chevrolet|     gas|   std|  two|hatchback|  fwd|    three| 48|5100|      47|     53| 5151|
|     mazda|     gas|   std|  two|hatchback|  fwd|     four| 68|5000|      30|     31| 5195|
|    toyota|     gas|   std|  two|hatchback|  fwd|     four| 62|4800|      35|     39| 5348|
|mitsubishi|     gas|   std|  two|hatchback|  fwd|     four| 68|5500|      37|     41| 5389|
|     honda|     gas|   std|  two|hatchback|  fwd|     four| 60|5500|      38|     42| 5399|
|    nissan|     gas|   std|  two|    sedan|  fwd|     four| 69|5200|      31|     37| 5499|
|     dodge|     gas|   std|  two|hatchback|  fwd|     four| 68|5500| 

In [7]:
autoDf1.printSchema()

root
 |-- MAKE: string (nullable = true)
 |-- FUELTYPE: string (nullable = true)
 |-- ASPIRE: string (nullable = true)
 |-- DOORS: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- DRIVE: string (nullable = true)
 |-- CYLINDERS: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- RPM: string (nullable = true)
 |-- MPG-CITY: string (nullable = true)
 |-- MPG-HWY: string (nullable = true)
 |-- PRICE: string (nullable = true)



#### Crear y trabajar con tablas temporales

In [8]:
autoDf1.createOrReplaceTempView("autos")
sqlContext.sql("select * from autos where hp > 200").show()

+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+
|   MAKE|FUELTYPE|ASPIRE|DOORS|       BODY|DRIVE|CYLINDERS| HP| RPM|MPG-CITY|MPG-HWY|PRICE|
+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+
|porsche|     gas|   std|  two|    hardtop|  rwd|      six|207|5900|      17|     25|32528|
|porsche|     gas|   std|  two|    hardtop|  rwd|      six|207|5900|      17|     25|34028|
| jaguar|     gas|   std|  two|      sedan|  rwd|   twelve|262|5000|      13|     17|36000|
|porsche|     gas|   std|  two|convertible|  rwd|      six|207|5900|      17|     25|37028|
+-------+--------+------+-----+-----------+-----+---------+---+----+--------+-------+-----+



In [9]:
%config Completer.use_jedi = False

In [11]:
autoDf1.count()

197

In [13]:
autoDf1.printSchema()

root
 |-- MAKE: string (nullable = true)
 |-- FUELTYPE: string (nullable = true)
 |-- ASPIRE: string (nullable = true)
 |-- DOORS: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- DRIVE: string (nullable = true)
 |-- CYLINDERS: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- RPM: string (nullable = true)
 |-- MPG-CITY: string (nullable = true)
 |-- MPG-HWY: string (nullable = true)
 |-- PRICE: string (nullable = true)

